In [1]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix,accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from sklearn.feature_selection import SelectKBest, f_classif,chi2
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

df = pd.read_csv('./train1.csv')
X_train = df.drop(['target','id'], axis=1)


In [2]:
y_train = df['target']

In [3]:
X_train,X_test,y_train,y_test = train_test_split(X_train, y_train, test_size=0.3, random_state=29173, stratify=y_train)

In [4]:
X_test.shape

(88863, 65)

In [5]:
"""df_test = pd.read_csv('test.csv')
X_test = df_test.drop('id',axis=1) 
test_ids = df_test['id']"""

"df_test = pd.read_csv('test.csv')\nX_test = df_test.drop('id',axis=1) \ntest_ids = df_test['id']"

In [6]:
categorical_cols = [col for col in X_train.columns if col.endswith('_cat')]
X_train[categorical_cols] = X_train[categorical_cols].astype('category')
X_test[categorical_cols] = X_test[categorical_cols].astype('category')
binary_cols = [col for col in X_train.columns if col.endswith('_bin')]
X_train[binary_cols] = X_train[binary_cols].astype(bool)
X_test[binary_cols] = X_test[binary_cols].astype(bool)

In [7]:
def simple_imputer(X_train, X_val, categorical_columns=None, numerical_columns=None):
    """
    Impute missing values in train and validation/test sets using only training data statistics.
    - categorical_columns: list of categorical columns to impute with mode
    - numerical_columns: list of numerical columns to impute with mean
    """
    X_train_imputed = X_train.copy()
    X_val_imputed = X_val.copy()

    # Categorical columns
    if categorical_columns:
        for col in categorical_columns:
            if col in X_train_imputed.columns:
                mode_value = X_train_imputed[col].mode(dropna=True)
                if not mode_value.empty:
                    mode_value = mode_value[0]
                    X_train_imputed[col].fillna(mode_value, inplace=True)
                    if col in X_val_imputed.columns:
                        X_val_imputed[col].fillna(mode_value, inplace=True)

    # Numerical columns
    if numerical_columns:
        for col in numerical_columns:
            if col in X_train_imputed.columns:
                mean_value = X_train_imputed[col].mean(skipna=True)
                if pd.notna(mean_value):  # ensure mean is valid
                    X_train_imputed[col].fillna(mean_value, inplace=True)
                    if col in X_val_imputed.columns:
                        X_val_imputed[col].fillna(mean_value, inplace=True)

    return X_train_imputed, X_val_imputed


In [8]:
num_cols = [col for col in X_train.columns if not col.endswith(('_cat', '_bin'))]

In [9]:
X_train_imputed,X_test_imputed = simple_imputer(X_train, X_test, categorical_cols,num_cols)

In [10]:
selected_features_anova_chi = ['ps_car_13', 'ps_reg_02', 'ps_car_12', 'feature4', 'ps_reg_03', 'feature2', 'ps_car_15', 'ps_ind_15', 'ps_reg_01', 'ps_ind_01', 'feature5', 'ps_car_14', 'feature7', 'ps_ind_03', 'ps_calc_01', 'ps_car_04_cat', 'ps_ind_05_cat', 'ps_car_11_cat', 'ps_car_06_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_ind_04_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_05_cat', 'ps_ind_17_bin', 'ps_ind_07_bin', 'ps_ind_06_bin', 'ps_ind_16_bin', 'ps_ind_08_bin', 'ps_ind_09_bin', 'ps_ind_12_bin']


In [11]:
X_train_selected_features = X_train_imputed[selected_features_anova_chi]
X_test_selected_features = X_test_imputed[selected_features_anova_chi]

In [12]:
significant_cat_features = ['ps_car_04_cat', 'ps_ind_05_cat', 'ps_car_11_cat', 'ps_car_06_cat', 'ps_car_01_cat', 'ps_car_02_cat', 'ps_ind_04_cat', 'ps_car_08_cat', 'ps_car_09_cat', 'ps_car_05_cat']

In [13]:
from sklearn.naive_bayes import CategoricalNB
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score


# Separate categorical and numerical columns
categorical_cols = [col for col in selected_features_anova_chi if col.endswith('_cat')]
binary_cols = [col for col in selected_features_anova_chi if col.endswith('_bin')]
numerical_cols = [col for col in selected_features_anova_chi 
                  if col not in categorical_cols + binary_cols]

# subset the data
X_train_sel = X_train_imputed[selected_features_anova_chi].copy()
X_val_sel = X_test_imputed[selected_features_anova_chi].copy()
y_train_sel = y_train.copy()
y_val_sel = y_test.copy()

# ensure categorical features are integer-encoded
from sklearn.preprocessing import LabelEncoder

for col in categorical_cols:
    le = LabelEncoder()
    X_train_sel[col] = le.fit_transform(X_train_sel[col].astype(str))
    X_val_sel[col] = le.transform(X_val_sel[col].astype(str))

# CategoricalNB expects all features to be discrete
# So we'll discretize numeric ones (optional, but helps model)
from sklearn.preprocessing import KBinsDiscretizer

kb = KBinsDiscretizer(n_bins=10, encode='ordinal', strategy='quantile')
X_train_sel[numerical_cols] = kb.fit_transform(X_train_sel[numerical_cols])
X_val_sel[numerical_cols] = kb.transform(X_val_sel[numerical_cols])

# Train Categorical Naive Bayes
cat_nb = CategoricalNB()
cat_nb.fit(X_train_sel, y_train_sel)

# Predict
y_pred = cat_nb.predict(X_val_sel)
y_proba = cat_nb.predict_proba(X_val_sel)[:, 1]

# Evaluate
print("Categorical NB Accuracy:", accuracy_score(y_val_sel, y_pred))
print("Categorical NB F1 Score:", f1_score(y_val_sel, y_pred, average='weighted'))
print("Categorical NB AUROC:", roc_auc_score(y_val_sel, y_proba))


d:\ML project\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:296: FutureWarning: The current default behavior, quantile_method='linear', will be changed to quantile_method='averaged_inverted_cdf' in scikit-learn version 1.9 to naturally support sample weight equivalence properties by default. Pass quantile_method='averaged_inverted_cdf' explicitly to silence this warning.
  warnings.warn(
d:\ML project\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:397: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 1 are removed. Consider decreasing the number of bins.
  warnings.warn(
d:\ML project\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:397: UserWarning: Bins whose width are too small (i.e., <= 1e-8) in feature 2 are removed. Consider decreasing the number of bins.
  warnings.warn(
d:\ML project\.venv\Lib\site-packages\sklearn\preprocessing\_discretization.py:397: UserWarning: Bins whose width are too small (i.e., <

Categorical NB Accuracy: 0.9074417924220429
Categorical NB F1 Score: 0.9077138411196586
Categorical NB AUROC: 0.6206718046120854
